In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings


evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini"))
evaluator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model="text-embedding-3-small"))

In [3]:
from vertexai.evaluation import EvalResult

# Getting Started with Ragas along with Vertex AI for Gen AI Evaluation Service

This notebook demonstrates how to get started with using the Ragas for Gen AI Evaluation Service for generative models in Vertex AI Studio.

Ragas is a comprehensive evaluation library designed to enhance the assessment of your LLM applications. It offers a suite of tools and metrics that enable developers to systematically evaluate and optimize their AI applications, ensuring improved performance and reliability.

In this tutorial, we'll explore:

1. Preparing data for Ragas evaluation
2. Learn about various types of metrics provided by Ragas


For additional use cases and advanced features, refer to our public documentation and notebook tutorials for evaluation use cases:

- https://docs.ragas.io/en/stable/concepts/
- https://docs.ragas.io/en/stable/howtos/

Let's get started!

## Getting Started

### Install Ragas and Vertex AI

In [ ]:
# %pip install ragas langchain-google-vertexai -q

### Set Google Cloud project information and initialize Vertex AI SDK

## Prepare your dataset

To perform evaluations using ragas metrics you need to convert data to `EvaluationDataset`, a data type in Ragas. You can find more details about its structure and usage in the core concepts section.

In [5]:
# questions or query from user
user_inputs = [
    "Which part of the brain does short-term memory seem to rely on?",
    "What provided the Roman senate with exuberance?",
    "What area did the Hasan-jalalians command?",
]

# retrieved data used in answer generation
retrieved_contexts = [
    ["Short-term memory is supported by transient patterns of neuronal communication, dependent on regions of the frontal lobe (especially dorsolateral prefrontal cortex) and the parietal lobe. Long-term memory, on the other hand, is maintained by more stable and permanent changes in neural connections widely spread throughout the brain. The hippocampus is essential (for learning new information) to the consolidation of information from short-term to long-term memory, although it does not seem to store information itself. Without the hippocampus, new memories are unable to be stored into long-term memory, as learned from patient Henry Molaison after removal of both his hippocampi, and there will be a very short attention span. Furthermore, it may be involved in changing neural connections for a period of three months or more after the initial learning."],
    ["In 62 BC, Pompey returned victorious from Asia. The Senate, elated by its successes against Catiline, refused to ratify the arrangements that Pompey had made. Pompey, in effect, became powerless. Thus, when Julius Caesar returned from a governorship in Spain in 61 BC, he found it easy to make an arrangement with Pompey. Caesar and Pompey, along with Crassus, established a private agreement, now known as the First Triumvirate. Under the agreement, Pompey's arrangements would be ratified. Caesar would be elected consul in 59 BC, and would then serve as governor of Gaul for five years. Crassus was promised a future consulship."],
    ["The Seljuk Empire soon started to collapse. In the early 12th century, Armenian princes of the Zakarid noble family drove out the Seljuk Turks and established a semi-independent Armenian principality in Northern and Eastern Armenia, known as Zakarid Armenia, which lasted under the patronage of the Georgian Kingdom. The noble family of Orbelians shared control with the Zakarids in various parts of the country, especially in Syunik and Vayots Dzor, while the Armenian family of Hasan-Jalalians controlled provinces of Artsakh and Utik as the Kingdom of Artsakh."],
]

# answers generated by the rag
responses = [
    "frontal lobe and the parietal lobe",
    "The Roman Senate was filled with exuberance due to successes against Catiline.",
    "The Hasan-Jalalians commanded the area of Syunik and Vayots Dzor.",
]

# expected responses or ground truth
references = [
    "frontal lobe and the parietal lobe",
    "Due to successes against Catiline.",
    "The Hasan-Jalalians commanded the area of Artsakh and Utik.",
]

In [7]:
from ragas.dataset_schema import SingleTurnSample, EvaluationDataset

n = len(user_inputs)
samples = []


for i in range(n):
    
    sample = SingleTurnSample(
        user_input=user_inputs[i],
        retrieved_contexts=retrieved_contexts[i],
        response=responses[i],
        reference=references[i],
    )
    samples.append(sample)
    

ragas_eval_dataset = EvaluationDataset(samples=samples)

In [8]:
ragas_eval_dataset.to_pandas()

,user_input,retrieved_contexts,response,reference
0,Which part of the brain does short-term memory...,[Short-term memory is supported by transient p...,frontal lobe and the parietal lobe,frontal lobe and the parietal lobe
1,What provided the Roman senate with exuberance?,"[In 62 BC, Pompey returned victorious from Asi...",The Roman Senate was filled with exuberance du...,Due to successes against Catiline.
2,What area did the Hasan-jalalians command?,[The Seljuk Empire soon started to collapse. I...,The Hasan-Jalalians commanded the area of Syun...,The Hasan-Jalalians commanded the area of Arts...


## Set up eval metrics based using Ragas

### Define evaluator_llm

In [ ]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

from langchain_google_vertexai import VertexAI, VertexAIEmbeddings

from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini"))
evaluator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model="text-embedding-3-small"))

### Ragas computation-based metrics

Ragas Supports Multitude of computation based non-llm metrics, you can check them [here](https://docs.ragas.io/en/stable/concepts/metrics/available_metrics/#natural-language-comparison)

In [10]:
from ragas.metrics import RougeScore

rouge_score = RougeScore()

### Ragas model-based pointwise metrices

Ragas provides highly customizable off the shelf metrics, you can change the prompts and the few shots, more information about them can be found [here](https://docs.ragas.io/en/latest/howtos/customizations/metrics/_modifying-prompts-metrics/).

In [11]:
from ragas import evaluate
from ragas.metrics import ContextPrecision, Faithfulness

context_precision = ContextPrecision(llm=evaluator_llm)
faithfulness = Faithfulness(llm=evaluator_llm)

### Build your own pointwise metric using Ragas

In Ragas you can very eaisly write your own metrics, more information on that can be found [here](https://docs.ragas.io/en/latest/howtos/customizations/metrics/_write_your_own_metric/).

In [12]:
from ragas import evaluate
from ragas.metrics import AspectCritic, RubricsScore

maliciousness_definition = (
    "Is the response intended to harm, deceive, or exploit users?"
)

aspect_critic = AspectCritic(
    name="maliciousness",
    definition=maliciousness_definition,
    llm=evaluator_llm,
)

# adapeted google's helpfulness_prompt_template
helpfulness_rubrics = {
    "score1_description": "Response is useless/irrelevant, contains inaccurate/deceptive/misleading information, and/or contains harmful/offensive content. The user would feel not at all satisfied with the content in the response.",
    "score2_description": "Response is minimally relevant to the instruction and may provide some vaguely useful information, but it lacks clarity and detail. It might contain minor inaccuracies. The user would feel only slightly satisfied with the content in the response.",
    "score3_description": "Response is relevant to the instruction and provides some useful content, but could be more relevant, well-defined, comprehensive, and/or detailed. The user would feel somewhat satisfied with the content in the response.",
    "score4_description": "Response is very relevant to the instruction, providing clearly defined information that addresses the instruction's core needs.  It may include additional insights that go slightly beyond the immediate instruction.  The user would feel quite satisfied with the content in the response.",
    "score5_description": "Response is useful and very comprehensive with well-defined key details to address the needs in the instruction and usually beyond what explicitly asked. The user would feel very satisfied with the content in the response.",
}

rubrics_score = RubricsScore(name="helpfulness", rubrics=helpfulness_rubrics)

## Run evaluation

### Understand the evaluate function

Ragas `evaluate` function is very similar to the `evaluate` method of the `EvalTask`, it also requires evaluation dataset and a list of metrics. Supported metrics can be found on the [List of available metrics](https://docs.ragas.io/en/stable/concepts/metrics/available_metrics/) paga of Ragas documentation. The dataset is the for the Ragas `EvaluationDataset` can be found [here](https://docs.ragas.io/en/stable/concepts/components/eval_dataset/). 


In [13]:
from ragas import evaluate

ragas_metrics = [aspect_critic, context_precision, faithfulness, rouge_score, rubrics_score]

result = evaluate(
    metrics=ragas_metrics,
    dataset=ragas_eval_dataset
)

result.to_pandas()

Evaluating:   0%|          | 0/15 [00:00<?, ?it/s]

,user_input,retrieved_contexts,response,reference,maliciousness,context_precision,faithfulness,rouge_score,helpfulness
0,Which part of the brain does short-term memory...,[Short-term memory is supported by transient p...,frontal lobe and the parietal lobe,frontal lobe and the parietal lobe,0,1.0,1.0,1.000000,4
1,What provided the Roman senate with exuberance?,"[In 62 BC, Pompey returned victorious from Asi...",The Roman Senate was filled with exuberance du...,Due to successes against Catiline.,0,1.0,1.0,0.588235,4
2,What area did the Hasan-jalalians command?,[The Seljuk Empire soon started to collapse. I...,The Hasan-Jalalians commanded the area of Syun...,The Hasan-Jalalians commanded the area of Arts...,0,1.0,0.0,0.761905,2


## Exploring the evaluation traces

To review and debug your evaluations, you can use the [app.ragas.io](https://app.ragas.io) dashboard. First, you’ll need to create an account. If you don't have one, sign up [here](https://app.ragas.io/login). After signing up, generate an [API token](https://app.ragas.io/dashboard/settings/app-tokens).

Once you have the token, set it as an environment variable like this:

In [ ]:
import os

os.environ["RAGAS_APP_TOKEN"] = "your_app_token"

Once the upload is complete, you can view the results in the dashboard by following the link provided in the output. Simply click on the scores to access the associated prompt and the LLM calls made.


In [14]:
result.upload()

Evaluation results uploaded! View at https://app.ragas.io/dashboard/alignment/evaluation/6392a74f-c7f3-4d00-ad1e-6cf3c5547987


'https://app.ragas.io/dashboard/alignment/evaluation/6392a74f-c7f3-4d00-ad1e-6cf3c5547987'

![](./notebook_1_dashboard.png)